<a href="https://colab.research.google.com/github/AnnSenina/Python_for_CL/blob/main/notebooks/Python_10_BeautifulSoup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Скрейперы, краулеры, парсеры

Веб-скрейпинг = краулинг + парсинг (приблизительно!)

Сегодня веб-скрейпинг и парсинг используются как синонимы

В этой тетрадке мы поговорим о способах собрать свой датасет для исследований: откуда брать данные, как их собирать и как хранить.

Существует несколько библиотек(модулей) для работы с веб-страничками, сегодня мы будем использовать requests для доступа к веб-страничкам и Beautiful Soup для работы с содержимым html-документов

In [ ]:
!pip install requests #ставим модуль requests

In [ ]:
# ставим модуль beautifulsoup, самая последняя версия - четвертая
!pip install beautifulsoup4 

In [ ]:
# импортируем модули в тетрадку

import requests as rq

from bs4 import BeautifulSoup

# Как работать с веб-страничками

### Шаг 1. 

Создадим переменную ```url``` и сохраним в нее адрес какой-нибудь html-страницы: например, учебной страницы, созданной в Вышке

обратите внимание, что адрес прописываем в кавычках, как строку

In [ ]:
url = 'https://online.hse.ru/python-as-foreign/1/'

В модуле requests есть метод request.get(), который сохраняет ответ сервера на наш реквест. Мы применим его к переменной url, куда сохранен путь к странице. 
Сохраним результат в переменную page

In [ ]:
page = rq.get(url) 

print(page) # посмотрим на код ответа, если 200, все хорошо
print(type(page))

код 200 сообщает, что страница загружена успешно 
*(коды, начинающиеся с 2, обычно указывают на успешное выполнение операции, а коды, начинающиеся с 4 или 5, сообщают об ошибке)*

Узнать больше о кодах состояния HTTP  можно [по этой ссылке.](https://www.w3.org/Protocols/HTTP/1.1/draft-ietf-http-v11-spec-01#Status-Codes)

In [ ]:
# есть еще одна команда, чтобы получить код в виде числа:
print(page.status_code)
print(type(page.status_code))

Следующим шагом нужно получить доступ к текстовому содержимому веб-файлов.

Здесь нам поможет page.text 

In [ ]:
print(page.text)

In [ ]:
page.encoding = 'utf-8' # укажем кодировку и снова посмотрим
print(page.text)

### Шаг2

Поработаем с текстом на страничке

Мы получили текст страницы (со всеми html-тегами), однако его неудобно прочитать в таком виде. 

Здесь нам понадобится Beautiful Soup, модуль для html-парсинга: он сделает текст веб-страницы, извлеченный с помощью Requests, более читаемым, потому что создает дерево синтаксического разбора из проанализированных тэгов.

In [ ]:
soup = BeautifulSoup(page.text, features="html.parser") #сохраним результат в переменную soup

In [ ]:
print(soup.prettify()) # показывает нашу страницу в красивом виде

### Шаг3 
время доставать тэги - но сначала...

### Немного про html

Перейдем на сайт [w3schools.com](https://www.w3schools.com/Html/) и откроем раздел Try it yourself.

Задание 1: 

1.   внутри тэгов h1 напишите ваши Ф.И.О.

2.   напишите подзаголовок "О себе", добавив тэги h2

3.   внутри тэгов p напишите "Я учусь создавать html-страницы."


Задание 2

Добавим таблицу

<table border="1">
<tr>
<th>Фамилия</th>
<th>Имя</th>
<th>Возраст</th>
</tr>
<tr>
<td></td>
<td></td>
<td></td>
</tr>
</table>

1. Заполните строку таблицы

2. Добавьте еще один столбец и заполните его

###Бонус! pandas и сам умеет собирать таблицы с сайтов, без rq, BeautifulSoup

In [ ]:
import pandas as pd

banks = pd.read_html('https://cbr.ru/currency_base/daily/')
print(banks) # возвращается список датафреймов - потому что вдруг таблиц на сайте несколько

In [193]:
data = pd.DataFrame(banks[0]) # 0 объект списка - наша едиснтвенная табличка на странице
data.to_csv("banks.csv")

In [ ]:
banks.to_csv("banks.csv")

###Вернемся к тегам:

предыдущие шаги со страницей позволили привести веб-страничку к виду, где содержание каждого тега написано с новой строки. 

Некоторые теги полезны для конкретной задачи (там текст), некоторые - не очень (например, мета-данные, картинки и тд)

Извлечь одинаковые теги со страницы можно с помощью метода find_all(). Он похож на метод регулярок, с которым мы работали: он вернет все экземпляры данного тега в документе. Нужно прописать в скобках метода нужный тег. 

### Самые популярные теги:
    <h1-h6> - заголовки
    <div> для целых "блоков" странички
    <li> список с перечислением
    <p> для текста
    <a> для гиперссылок
    <img> для изображений

In [ ]:
soup.find_all("head") # тег нужно записать без треугольных скобочек
#print(soup.find_all("head")) # для PyCharm

# попробуйте теги body, title, a и др.

А так можно достать нужные части тега (напимер, текст)

In [ ]:
for x in soup.find_all('a'):
    print(x.text)

In [ ]:
# Весь текст на страничке за раз можно достать еще и так
print(soup.text)

## Как создать корпус

Итак, мы определили нужные теги и напарсили необходимые данные. Пора сохранить их в файл. 

In [ ]:
# в .txt

with open("HP.txt", "w") as file:
    for x in soup.find_all("a"):
        file.write(x.text)
        
# необязательно, но можем скорретировать информацию, которую записываем

In [ ]:
#   или в .csv

names = []
for x in soup.find_all("a"):
  names.append(x.text)

links = []
for link in soup.find_all("a"):
  links.append(link.get('href')) # вот так можно достать ссылку из тега a

print(names)
print(links)

In [ ]:
data = []

for i in range(len(names)): # списки равной длины, переберем поиндексно
  l = []
  l.append(names[i])
  l.append('https://online.hse.ru/python-as-foreign/1/' + links[i])
  data.append(l)

print(data)

In [ ]:
import pandas as pd

df = pd.DataFrame(data, columns=["name", "url"])

df
# print(df) #для PyCharm

In [ ]:
# сохраняем
df.to_csv("HP.csv")

## Практика

давайте попарсим другой адрес и вытащим оттуда весь текст

In [ ]:
url = "https://en.wikipedia.org/wiki/Welsh_Corgi"

In [ ]:
# ваш код ниже
# передаем в rq



In [ ]:
# передаем в BeautifulSoup


In [ ]:
# сохраняем


##Бонус (в т.ч. для домашнего задания)

Как заставить код спарсить несколько страничек

In [ ]:
url = 'https://nplus1.ru/' # сохраняем
page = rq.get(url) # загружаем страницу по ссылке
print(page.status_code)  # 200 - страница загружена

In [ ]:
soup = BeautifulSoup(page.text, features="html.parser")
print(soup.prettify())

шаг 1 - соберем с главной страницы ссылки на те, которые хотим спарсить

In [ ]:
urls = []

for link in soup.find_all('a'):
    urls.append(link.get('href'))

urls
#print(urls)

# много лишних ссылок и повторов, оставим только ссылки на новости

In [ ]:
full_urls = []

for link in soup.find_all('a'):
  if 'news' in link.get('href') and 'https://nplus1.ru' in link.get('href') and link.get('href') not in full_urls:
    full_urls.append(link.get('href')) 

full_urls
# print(full_urls)

In [ ]:
# первая ссылка - попала в выдачу не совсем корректно (тематически нам не подходит), ее можно удалить или позже указать срез ссылок для обработки, пропустив ее
# удалим сразу

del full_urls[0]
print(full_urls)

шаг 2 - посмотрим, что можно спарсить со страницы 1 любой новости

In [ ]:
url0 = full_urls[0]

page0 = rq.get(url0)
soup0 = BeautifulSoup(page0.text)
print(soup0.prettify())

In [ ]:
# случай сложный: самая интересная информация хранится в теге meta

soup0.find_all('meta')
#print(soup0.find_all('meta'))

Соберем информацию об авторе, дате публикации, заголовке 
Какие атрибуты нам нужны? 

author, datePublished, og:title

In [ ]:
# атрибуты прописаны внутри тега meta, как их извлечь?
# передадим нужный атрибут как словарь 

author = soup0.find_all('meta', {'name' : 'author'})[0].attrs['content'] # вызовем автора по ключу (content) и сохраним в переменную
date = soup0.find_all('meta', {'itemprop' : 'datePublished'})[0].attrs['content']
title = soup0.find_all('meta', {'property' : 'og:title'})[0].attrs['content']

print(author, date, title)

Извлечем текст новости, используя тэг p с атрибутом mb-6

In [ ]:
text_list = soup0.find_all('p', {'class' : 'mb-6'})
text_list
#print(text_list)

In [ ]:
text = []
for i in text_list:
  text.append(i.text)
text
#print(text)

In [ ]:
# слегка поправим текст:
final_text = ' '.join(text)
final_text = final_text.replace('\xa0', ' ')
final_text
#print(final_text)

шаг 3: напишем функцию, которая будет идти по ссылкам из full_urls и парсить каждую страницу

In [ ]:
def GetNews(url0):
  page0 = rq.get(url0)
  soup0 = BeautifulSoup(page0.text, features="html.parser")
  author = soup0.find_all('meta', {'name' : 'author'})[0].attrs['content'] 
  date = soup0.find_all('meta', {'itemprop' : 'datePublished'})[0].attrs['content']
  title = soup0.find_all('meta', {'property' : 'og:title'})[0].attrs['content']
  text_list = soup0.find_all('p', {'class' : 'mb-6'})
  text = []
  for i in text_list:
    text.append(i.text)
  final_text = ' '.join(text)
  final_text = final_text.replace('\xa0', ' ')
  return url0, author, date, title, final_text

In [ ]:
news = [] # список с новостями

for link in full_urls:
  new = GetNews(link)
  news.append(new)

In [ ]:
df = pd.DataFrame(news)
df.head()

In [ ]:
df.columns = ['link', 'author', 'date', 'title', 'text']

In [ ]:
df.head(1)

In [ ]:
df.to_excel('nplus_news.xlsx')

### Как парсить сайты, не раздражая сервера

Выше мы пробовали скрейпить сайт циклом. Скорость такого скрейпинга ограничена только скоростью сети и быстродействием компьютеров. То есть может быть довольно высокой

Это нагрузка на сервера, и иногда владельцы блокируют скрейперов

In [ ]:
%%time 

for i in range(10): # 10 раз запроси одну и ту же страницу
    rq.get('https://quotes.toscrape.com')

1. Использовать sleep из модуля time

In [ ]:
import time

In [ ]:
%%time 

for i in range(10):
    rq.get('https://quotes.toscrape.com')
    time.sleep(1) # ждем 1 секунду после каждого шага

2. Мимикрировать под человека с помощью fake_useragent

Допустим, вы решили парсить мемы:

In [ ]:
page_link = 'https://knowyourmeme.com/memes/all/page/1'
response = rq.get(page_link)

In [ ]:
soup = BeautifulSoup(response.text)
print(soup.find("table")) # спасим таблицу с мемами

In [ ]:
print(response.status_code)

In [ ]:
print(response.text)
# для нас запрещено!
# сервер видит, что к нему пришла программа, а не человек

In [ ]:
!pip install fake_useragent

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 3.8 MB/s 


In [ ]:
from fake_useragent import UserAgent
print(UserAgent().chrome)

In [ ]:
page_link = 'https://knowyourmeme.com/memes/all/page/1'
response = rq.get(page_link, headers={'User-Agent': UserAgent().chrome})

In [ ]:
print(response.status_code)

In [ ]:
print(response.text)

In [ ]:
soup = BeautifulSoup(response.text)
soup.find('table')

3. В крайних случаях можно использовать тор/прокси-серверы для смены IP

* [Раз](https://habr.com/ru/company/ods/blog/346632/#22-tor---syn-odina)
* [Два](https://stackoverflow.com/questions/30286293/make-requests-using-python-over-tor)


## Полезный совет про try / except!

Когда парсите много страниц разом, велик риск получить ошибки (какая-то страница не ответила, сервер начал нас блокировать и т.д.)

Используйте try / except

In [ ]:
for i in range(0,10):
    print(100/i)

print('Все в порядке!')

In [ ]:
for i in range(0,10):
    try:
        print(100/i)
    except: 
        print(f'с числом {i} не работает')

print('Все в порядке!')

Как использовать при парсинге:

In [ ]:
unis_to_parse = ['hse', 'itmo', 'msu', 'hsе', 'spbu']

In [ ]:
for address in unis_to_parse:
    page_response = rq.get(f'https://{address}.ru')
    current_html = BeautifulSoup(page_response.text)
    print(current_html.title)

In [ ]:
for address in unis_to_parse:
    try:
        page_response = rq.get(f'https://{address}.ru')
        current_html = BeautifulSoup(page_response.text)
        print(current_html.title)
    except:
        print(f'С сайтом https://{address}.ru не работает')

# hse - во втором случае использована е кириллицей

# Полезные ссылки

Что почитать об использовании данных
- [как устроен сбор данных](https://en.wikipedia.org/wiki/Web_scraping)


- закон об авторском праве ([в деталях](http://www.consultant.ru/document/cons_doc_LAW_64629/0b318126c43879a845405f1fb1f4342f473a1eda/), [вкратце](https://ru.wikipedia.org/wiki/%D0%90%D0%B2%D1%82%D0%BE%D1%80%D1%81%D0%BA%D0%BE%D0%B5_%D0%BF%D1%80%D0%B0%D0%B2%D0%BE_%D0%B2_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8))
- [закон о персональных данных](http://www.consultant.ru/document/cons_doc_LAW_61801/)
- [типы лицензирования данных](https://docs.github.com/en/repositories/managing-your-repositorys-settings-and-features/customizing-your-repository/licensing-a-repository)
- [FAIR data](https://en.wikipedia.org/wiki/FAIR_data)
- [OpenData](https://en.wikipedia.org/wiki/Open_data)



Гайды и туториалы

- [документация requests и быстрый гайд](https://requests.readthedocs.io/en/master/user/quickstart/)


- [документация Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

- [text-only](https://sjmulder.nl/en/textonly.html) веб-сайты, чтобы легко начать парсить



- [здесь](https://www.york.ac.uk/teaching/cws/wws/webpage1.html) можно почитать про структуру html подробнее


- [здесь](https://www.w3schools.com/html/html_examples.asp) еще и потренироваться в режиме онлайн (с этой ссылки мы начали занятие)

Чем парсить соцсети (не исчерпывающий список)

(стало сложнее, в России 1 и 2 ссылка не откроются без VPN...) 

- [Twitter](https://developer.twitter.com/en/docs/twitter-api/tools-and-libraries/v2)
- [Meta](https://developers.facebook.com/docs/graph-api/)
- [VK](https://vk-api.readthedocs.io/en/latest/), [положения о прайваси](https://vk.com/dev/uprivacy)

## Посмотрите дополнительные тетрадки! 

[Здесь](https://github.com/AnnSenina/Python_for_CL/tree/main/notebooks/дополнительные%20тетрадки) появились новые тетрадки про скрейпинг